# "Рекомендательные системы" -- домашнее задание

## Автор материала: доцент ФКН НИУ ВШЭ Дмитрий Игнатов
Можно использовать материал в любых целях, кроме коммерческих. Примеры составлены по мотивам главы 2 книги Т.Сегаран "Программируем коллективный разум" http://www.symbol.ru/alphabet/613828.html

### Срок выполнения: см. пост в Piazza (день дедлайна включительно)




## Словарь с предпочтениями пользователи-фильмы

In [1]:
# Словарь кинокритиков и выставленных ими оценок для небольшого набора
# данных о фильмах
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [2]:
critics['Toby']

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}

In [3]:
critics['Lisa Rose']['Lady in the Water']


2.5

In [6]:
#вычисление расстояния Евклида
from math import sqrt
sqrt(pow(5-4,2)+pow(4-1,2))


3.1622776601683795

In [7]:
#вычисление сходства
1/(1+sqrt(pow(5-4.5,2)+pow(5-5,2)))


0.6666666666666666

In [8]:
from numpy import exp

In [9]:
exp(-0.3*sqrt(pow(5-4.5,2)+pow(5-5,2)))


0.86070797642505781

In [13]:
from math import sqrt
# Возвращает сходство person1 и person2 на основе расстояния
def sim_distance(prefs,person1,person2):
# Получить список предметов, оцененных обоими
    si={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
# Если нет ни одной общей оценки, вернуть 0
    if len(si)==0: return 0
# Сложить квадраты разностей/
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2)
    for item in prefs[person1] if item in prefs[person2]])
    return 1/(1+sum_of_squares)


In [14]:
from math import sqrt
# Возвращает сходство person1 и person2 на основе расстояния
def sim_kernel(prefs,person1,person2,alpha=0.3):
# Получить список предметов, оцененных обоими
    si={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
# Если нет ни одной общей оценки, вернуть 0
    if len(si)==0: return 0
# Сложить квадраты разностей
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2)
    for item in prefs[person1] if item in prefs[person2]])
    return exp(-alpha*sum_of_squares)

In [15]:
sim_kernel(critics, 'Lisa Rose','Toby')

0.34993774911115533

In [16]:
sim_distance(critics, 'Lisa Rose','Toby')

0.2222222222222222

In [17]:
# Возвращает коэффициент корреляции Пирсона между p1 и p2
def sim_pearson(prefs,p1,p2):
# Получить список предметов, оцененных обоими
    si={}
    for item in prefs[p1]: 
        if item in prefs[p2]: si[item]=1

    # Если нет ни одной общей оценки, вернуть 0
    if len(si)==0: return 0

    # Количество соместно оцененных фильм
    n=len(si)
  
    # Вычислить сумму всех предпочтений
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
  
    # Вычислить сумму квадратов
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])	
  
    # Вычислить сумму произведений
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
  
    # Вычислить коэффициент Пирсона
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: return 0

    r=num/den

    return r

In [18]:
sim_pearson(critics,'Lisa Rose','Gene Seymour'), sim_distance(critics, 'Lisa Rose','Gene Seymour')



(0.39605901719066977, 0.14814814814814814)

## Ранжирование критиков

In [19]:
# Возвращает список наилучших соответствий для человека из словаря prefs.
# Количество результатов в списке и функция подобия – необязательные
# параметры.
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    scores=[(similarity(prefs,person,other),other)
    for other in prefs if other!=person]
    # Отсортировать список по убыванию оценок
    scores.sort( )
    scores.reverse( )
    return scores[0:n]


In [20]:
topMatches(critics,'Toby',n=3, similarity=sim_distance)

[(0.3076923076923077, 'Mick LaSalle'),
 (0.2857142857142857, 'Michael Phillips'),
 (0.23529411764705882, 'Claudia Puig')]

In [21]:
topMatches(critics,'Toby',n=3, similarity=sim_kernel)

[(0.50915642060754918, 'Mick LaSalle'),
 (0.47236655274101469, 'Michael Phillips'),
 (0.37719235356315695, 'Claudia Puig')]

## Рекомендация фильмов (User-based подход)

In [22]:
# Получить рекомендации для заданного человека, пользуясь взвешенным средним
# оценок, данных всеми остальными пользователями
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
    # сравнивать меня с собой же не нужно
        if other==person: continue
        sim=similarity(prefs,person,other)
    # игнорировать нулевые и отрицательные оценки
        if sim<=0: continue
        for item in prefs[other]:
    # оценивать только фильмы, которые я еще не смотрел
            if item not in prefs[person] or prefs[person][item]==0:
    # Коэффициент подобия * Оценка
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
        # Сумма коэффициентов подобия
                simSums.setdefault(item,0)
                simSums[item]+=sim
    # Создать нормированный список
    rankings=[(total/simSums[item],item) for item,total in totals.items( )]
    # Вернуть отсортированный список
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [23]:
getRecommendations(critics,'Toby')


[(3.3477895267131013, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.5309807037655645, 'Just My Luck')]

In [24]:
getRecommendations(critics,'Toby', sim_kernel)

[(3.5469111370529642, 'The Night Listener'),
 (2.7296870804186324, 'Lady in the Water'),
 (2.5187995455498609, 'Just My Luck')]

## Сходство предметов

Как заменить

{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5}}

на

{'Lady in the Water':{'Lisa Rose':2.5,'Gene Seymour':3.0},
'Snakes on a Plane':{'Lisa Rose':3.5,'Gene Seymour':3.5}}?


In [25]:
def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
    # Обменять местами человека и предмет
    
            result[item][person]=prefs[person][item]
    return result

In [26]:
movies=transformPrefs(critics)


In [27]:
movies

{'Just My Luck': {'Claudia Puig': 3.0,
  'Gene Seymour': 1.5,
  'Lisa Rose': 3.0,
  'Mick LaSalle': 2.0},
 'Lady in the Water': {'Gene Seymour': 3.0,
  'Jack Matthews': 3.0,
  'Lisa Rose': 2.5,
  'Michael Phillips': 2.5,
  'Mick LaSalle': 3.0},
 'Snakes on a Plane': {'Claudia Puig': 3.5,
  'Gene Seymour': 3.5,
  'Jack Matthews': 4.0,
  'Lisa Rose': 3.5,
  'Michael Phillips': 3.0,
  'Mick LaSalle': 4.0,
  'Toby': 4.5},
 'Superman Returns': {'Claudia Puig': 4.0,
  'Gene Seymour': 5.0,
  'Jack Matthews': 5.0,
  'Lisa Rose': 3.5,
  'Michael Phillips': 3.5,
  'Mick LaSalle': 3.0,
  'Toby': 4.0},
 'The Night Listener': {'Claudia Puig': 4.5,
  'Gene Seymour': 3.0,
  'Jack Matthews': 3.0,
  'Lisa Rose': 3.0,
  'Michael Phillips': 4.0,
  'Mick LaSalle': 3.0},
 'You, Me and Dupree': {'Claudia Puig': 2.5,
  'Gene Seymour': 3.5,
  'Jack Matthews': 3.5,
  'Lisa Rose': 2.5,
  'Mick LaSalle': 2.0,
  'Toby': 1.0}}

In [28]:
topMatches(movies,'Snakes on a Plane',5, sim_pearson)

[(0.7637626158259785, 'Lady in the Water'),
 (0.11180339887498941, 'Superman Returns'),
 (-0.3333333333333333, 'Just My Luck'),
 (-0.5663521139548527, 'The Night Listener'),
 (-0.6454972243679047, 'You, Me and Dupree')]

In [30]:
getRecommendations(movies,'The Night Listener', sim_distance)

[(3.182634730538922, 'Toby')]

## Коллаборативная фильтрация по сходству объектов (Item-based collaborative filtering)

In [31]:
def calculateSimilarItems(prefs,n=10):
    # Создать словарь, содержащий для каждого образца те образцы, которые
    # больше всего похожи на него.
    result={}
    # Обратить матрицу предпочтений, чтобы строки соответствовали образцам
    itemPrefs=transformPrefs(prefs)
    c=0
    for item in itemPrefs:
    # Обновление состояния для больших наборов данных
        c+=1
        if c%100==0: print "%d / %d" % (c,len(itemPrefs))
    # Найти образцы, максимально похожие на данный
        scores=topMatches(itemPrefs,item,n=n,similarity=sim_distance)
        result[item]=scores
    return result

In [32]:
itemsim=calculateSimilarItems(critics)
itemsim

{'Just My Luck': [(0.2222222222222222, 'Lady in the Water'),
  (0.18181818181818182, 'You, Me and Dupree'),
  (0.15384615384615385, 'The Night Listener'),
  (0.10526315789473684, 'Snakes on a Plane'),
  (0.06451612903225806, 'Superman Returns')],
 'Lady in the Water': [(0.4, 'You, Me and Dupree'),
  (0.2857142857142857, 'The Night Listener'),
  (0.2222222222222222, 'Snakes on a Plane'),
  (0.2222222222222222, 'Just My Luck'),
  (0.09090909090909091, 'Superman Returns')],
 'Snakes on a Plane': [(0.2222222222222222, 'Lady in the Water'),
  (0.18181818181818182, 'The Night Listener'),
  (0.16666666666666666, 'Superman Returns'),
  (0.10526315789473684, 'Just My Luck'),
  (0.05128205128205128, 'You, Me and Dupree')],
 'Superman Returns': [(0.16666666666666666, 'Snakes on a Plane'),
  (0.10256410256410256, 'The Night Listener'),
  (0.09090909090909091, 'Lady in the Water'),
  (0.06451612903225806, 'Just My Luck'),
  (0.05333333333333334, 'You, Me and Dupree')],
 'The Night Listener': [(0.28

In [33]:
def getRecommendedItems(prefs,itemMatch,user):
    userRatings=prefs[user]
    scores={}
    totalSim={}

    # Цикл по образцам, оцененным данным пользователем
    for (item,rating) in userRatings.items():
        
    # Цикл по образцам, похожим на данный
        for (similarity,item2) in itemMatch[item]:
    # Пропускаем, если пользователь уже оценивал данный образец
            if item2 in userRatings: continue
        # Взвешенная суммы оценок, умноженных на коэффициент подобия
            scores.setdefault(item2,0)
            scores[item2]+=similarity*rating
        # Сумма всех коэффициентов подобия
            totalSim.setdefault(item2,0)
            totalSim[item2]+=similarity
            if totalSim[item2]==0: totalSim[item2]=0.0000001 # чтобы избежать деления на ноль
    # Делим каждую итоговую оценку на взвешенную сумму, чтобы вычислить
    # среднее
    rankings=[(score/totalSim[item],item) for item,score in scores.items( ) ]

    # Возвращает список rankings, отсортированный по убыванию
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [34]:
getRecommendedItems(critics,itemsim,'Toby')



[(3.182634730538922, 'The Night Listener'),
 (2.5983318700614575, 'Just My Luck'),
 (2.4730878186968837, 'Lady in the Water')]

## Рекомендация на данных MovieLens

Источник: http://grouplens.org/datasets/movielens/

In [36]:
def loadMovieLens(path='data/'):
# Получить названия фильмов
    movies={}
    for line in open(path+'/u.item'):
        (id,title)=line.split('|')[0:2]
        movies[id]=title
# Загрузить данные
    prefs={}
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
    return prefs

In [37]:
prefs=loadMovieLens( )
prefs['87']

{'2001: A Space Odyssey (1968)': 5.0,
 'Ace Ventura: Pet Detective (1994)': 4.0,
 'Addams Family Values (1993)': 2.0,
 'Addicted to Love (1997)': 4.0,
 'Adventures of Priscilla, Queen of the Desert, The (1994)': 3.0,
 'Adventures of Robin Hood, The (1938)': 5.0,
 'Air Force One (1997)': 3.0,
 'Air Up There, The (1994)': 3.0,
 'Alien (1979)': 4.0,
 'American President, The (1995)': 5.0,
 'Annie Hall (1977)': 4.0,
 'Apocalypse Now (1979)': 4.0,
 'Babe (1995)': 5.0,
 'Baby-Sitters Club, The (1995)': 2.0,
 'Back to the Future (1985)': 5.0,
 'Bad Boys (1995)': 4.0,
 'Bananas (1971)': 5.0,
 'Barcelona (1994)': 3.0,
 'Batman & Robin (1997)': 4.0,
 'Batman (1989)': 3.0,
 'Batman Returns (1992)': 3.0,
 'Big Green, The (1995)': 3.0,
 'Big Squeeze, The (1996)': 2.0,
 'Birdcage, The (1996)': 4.0,
 'Blade Runner (1982)': 4.0,
 'Blues Brothers, The (1980)': 5.0,
 'Boomerang (1992)': 3.0,
 'Boot, Das (1981)': 4.0,
 'Brady Bunch Movie, The (1995)': 2.0,
 'Braveheart (1995)': 4.0,
 'Bridge on the River

In [38]:
len(prefs['87'])

210

In [39]:
getRecommendations(prefs,'87')[0:30]

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, 'Star Kid (1997)'),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Saint of Fort Washington, The (1993)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Great Day in Harlem, A (1994)'),
 (5.0, 'Entertaining Angels: The Dorothy Day Story (1996)'),
 (5.0, 'Boys, Les (1997)'),
 (4.89884443128923, 'Legal Deceit (1997)'),
 (4.815019082242709, 'Letter From Death Row, A (1998)'),
 (4.7321082983941425, 'Hearts and Minds (1996)'),
 (4.696244466490867, 'Pather Panchali (1955)'),
 (4.652397061026758, 'Lamerica (1994)'),
 (4.538723693474813, 'Leading Man, The (1996)'),
 (4.535081339106103, 'Mrs. Dalloway (1997)'),
 (4.532337612572981, 'Innocents, The (1961)'),
 (4.527998574747079, 'Casablanca (1942)'),
 (4.510270149719864, 'Everest (1998)'),
 (4.493967755428439, 'Dangerous Beauty (1998)'),
 (4.485151301801342, 'Wallace & Gromit: The Best of Aardman Animation (1996)'),
 (4.463287461290222, 'Wrong Trousers, The (1993)'),
 (4.450979436941

In [40]:
itemsim=calculateSimilarItems(prefs,n=50)

100 / 1664
200 / 1664
300 / 1664
400 / 1664
500 / 1664
600 / 1664
700 / 1664
800 / 1664
900 / 1664
1000 / 1664
1100 / 1664
1200 / 1664
1300 / 1664
1400 / 1664
1500 / 1664
1600 / 1664


In [32]:
itemsim["What's Eating Gilbert Grape (1993)"]

[(1.0, 'Yankee Zulu (1994)'),
 (1.0, 'Woman in Question, The (1950)'),
 (1.0, 'Witness (1985)'),
 (1.0, "Wend Kuuni (God's Gift) (1982)"),
 (1.0, 'Vie est belle, La (Life is Rosey) (1987)'),
 (1.0, 'Vermont Is For Lovers (1992)'),
 (1.0, 'Van, The (1996)'),
 (1.0, 'Two or Three Things I Know About Her (1966)'),
 (1.0, 'Two Much (1996)'),
 (1.0, 'Two Deaths (1995)'),
 (1.0, 'Two Bits (1995)'),
 (1.0, 'Touki Bouki (Journey of the Hyena) (1973)'),
 (1.0, 'Total Eclipse (1995)'),
 (1.0, 'To Have, or Not (1995)'),
 (1.0, 'To Cross the Rubicon (1991)'),
 (1.0, 'Tigrero: A Film That Was Never Made (1994)'),
 (1.0, 'The Courtyard (1995)'),
 (1.0, 'Terror in a Texas Town (1958)'),
 (1.0, 'Talking About Sex (1994)'),
 (1.0, 'T-Men (1947)'),
 (1.0, 'Symphonie pastorale, La (1946)'),
 (1.0, 'Swept from the Sea (1997)'),
 (1.0, 'Stranger in the House (1997)'),
 (1.0, 'Stonewall (1995)'),
 (1.0, 'Squeeze (1996)'),
 (1.0, 'Spirits of the Dead (Tre passi nel delirio) (1968)'),
 (1.0, 'Somebody to Love

In [44]:
getRecommendedItems(prefs,itemsim,'87')[0:30]

[(5.0, "What's Eating Gilbert Grape (1993)"),
 (5.0, 'Vertigo (1958)'),
 (5.0, 'Usual Suspects, The (1995)'),
 (5.0, 'Toy Story (1995)'),
 (5.0, 'Titanic (1997)'),
 (5.0, 'Sword in the Stone, The (1963)'),
 (5.0, 'Stand by Me (1986)'),
 (5.0, 'Sling Blade (1996)'),
 (5.0, 'Silence of the Lambs, The (1991)'),
 (5.0, 'Shining, The (1980)'),
 (5.0, 'Shine (1996)'),
 (5.0, 'Sense and Sensibility (1995)'),
 (5.0, 'Scream (1996)'),
 (5.0, 'Rumble in the Bronx (1995)'),
 (5.0, 'Rock, The (1996)'),
 (5.0, 'Robin Hood: Prince of Thieves (1991)'),
 (5.0, 'Reservoir Dogs (1992)'),
 (5.0, 'Police Story 4: Project S (Chao ji ji hua) (1993)'),
 (5.0, 'House of the Spirits, The (1993)'),
 (5.0, 'Fresh (1994)'),
 (5.0, 'Denise Calls Up (1995)'),
 (5.0, 'Day the Sun Turned Cold, The (Tianguo niezi) (1994)'),
 (5.0, 'Before the Rain (Pred dozhdot) (1994)'),
 (5.0, 'Assignment, The (1997)'),
 (5.0, '1-900 (1994)'),
 (4.875, "Ed's Next Move (1996)"),
 (4.833333333333333, 'Anna (1996)'),
 (4.8, 'Dark City 

# Задание 1. Сравнение методов коллаборативной фильтрации по сходству пользователей и по сходству объектов


1. Требуется реализовать вычисление ошибки [MAE](http://www.recsyswiki.com/wiki/Mean_absolute_error) и [RMSE](http://www.recsyswiki.com/wiki/Root_mean_square_error-mean-square_deviation) на тестовых данных [Movie Lens](http://grouplens.org/datasets/movielens/).  
В качестве данных обучения можно использовать файлы с расширением base, а тестирование качества провести на файле test: пары файлов u1.base и u1.test, ..., u5.base и u5.test. Каждая пара -- это разбиение 80%/20%  данных для всех пользователей $u$ на обучащие и тестовые данные.
2. Для каждого метода (user-based и item-based) постройте графики зависимости [MAE](http://www.recsyswiki.com/wiki/Mean_absolute_error) и [RMSE](http://www.recsyswiki.com/wiki/Root_mean_square_error-mean-square_deviation) от числа соседей (диапазон от 1 до 100 с разумным шагом).
3. Если качество предсказаний слишком низкое (MAE>2,0), то попробуйте формулы 2.6 и 2.7 из обзора http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf.
Можно использовать альтернативные формулы для исходной модели $r_{u,i} = k\sum\limits_{u^\prime \in U}\operatorname{sim}(u,u^\prime)r_{u^\prime, i} \mbox{ (случай user-based модели):}$
$$r_{u,i} = \frac{1}{N}\sum\limits_{u^\prime \in U}r_{u^\prime, i}$$
$$r_{u,i} = \bar{r_u} +  k\sum\limits_{u^\prime \in U} sim(u,u^\prime)(r_{u^\prime, i}-\bar{r_{u^\prime}} ) \mbox{, где } k =1/\sum_{u^\prime \in U}|\operatorname{sim}(u,u^\prime)|.$$
4. Сравните подходы на основе полученных результатов по аналогии с пунктами 1 и 2. 
5. Как изменяется величина MAE (RMSE) от числа выдаваемых рекомендаций (top-n): $n \in \{1,3,5,10,15,20,30,40,50,100\}$? 
6. Как Вы считаете, какие фильмы чаще рекомендуются -- популярные с высокими оценками или редкие (те, которые редко оцениваются) с высокими оценками?
7. Что делать, если соседей (то есть похожих на целевого пользователя или конкретный товар) мало? Нужно/можно ли как-то учитывать достоверность таких рекомендаций? 
8. *Необязательное подзадание.* Насколько различны списки из top-n рекомендаций. Попробуйте улучшить результаты подбором $\beta$ для минимизации MAE (RMSE) в гибридных рекомендациях в зависимости от числа соседей:
$$\beta\cdot r^{user-based}_{ui} + (1-\beta)\cdot r^{item-based}_{ui}, \mbox{ где } 0 \leq \beta \leq 1.$$ 

